In [300]:
from os import path, getenv
from mlrun import new_project

#project_name = '-'.join(filter(None, ['getting-started-iris', getenv('V3IO_USERNAME', None)]))
project_name = "parquez"
project_path = path.abspath('./')
project = new_project(project_name, project_path)
project.save()
print(f'Project path: {project_path}\nProject name: {project_name}')

Project path: /User/parquez
Project name: parquez


In [301]:
#print(project.yaml)

In [302]:
%env PYTHONPATH=./

env: PYTHONPATH=./


In [303]:
project.set_function("parquez.py", 'parquezrun', kind='job', image='aviaigz/parquez')

In [304]:
project.set_function("validate-input.py", 'validate', kind='job', image='aviaigz/parquez')

In [305]:
from mlrun import run_local, mount_v3io

In [306]:
project.func('parquezrun').apply(mount_v3io())
project.func('parquezrun').set_env('PYTHONPATH', project_path)
project.func('parquezrun').spec.service_account='mlrun-api'
project.func('validate').apply(mount_v3io())
project.func('validate').set_env('PYTHONPATH', project_path)

In [307]:
#project.func('parquezrun').run()
project.func('validate').run(params={'view_name':'view_name'
         ,'partition_by':'h'
         ,'partition_interval':'1h'
         ,'real_time_window':'1d'
         ,'historical_retention':'7d'
         ,'real_time_table_name':'faker'
         ,'config_path':'User/parquez/config/parquez.ini'})

[mlrun] 2020-06-14 12:37:18,900 artifact path is not defined or is local, artifacts will not be visible in the UI
[mlrun] 2020-06-14 12:37:18,918 starting run validate uid=7a4a14505ca34afda3baa2499e201937  -> http://10.192.248.103:8080
[mlrun] 2020-06-14 12:37:19,134 Job is running in the background, pod: validate-j8mnv
[mlrun] 2020-06-14 12:37:31,801 artifact path is not defined or is local, artifacts will not be visible in the UI
[mlrun] 2020-06-14 12:37:36,071 logging run results to: http://10.192.248.103:8080
[mlrun] 2020-06-14 12:37:36,123 {'view_name': 'view_name', 'partition_by': 'h', 'partition_interval': '1h', 'real_time_window': '1d', 'historical_retention': '7d', 'real_time_table_name': 'faker', 'config_path': 'User/parquez/config/parquez.ini'}
[mlrun] 2020-06-14 12:37:36,123 Starting to Parquezzzzzzzz
[mlrun] 2020-06-14 12:37:36,125 Namespace(config='User/parquez/config/parquez.ini', historical_retention='7d', partition_by='h', partition_interval='1h', real_time_table_name=

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
parquez,...9e201937,0,Jun 14 12:37:36,completed,validate,host=validate-j8mnvkind=jobowner=iguaziov3io_user=iguazio,,config_path=User/parquez/config/parquez.inihistorical_retention=7dpartition_by=hpartition_interval=1hreal_time_table_name=fakerreal_time_window=1dview_name=view_name,,


to track results use .show() or .logs() or in CLI: 
!mlrun get run 7a4a14505ca34afda3baa2499e201937 --project parquez , !mlrun logs 7a4a14505ca34afda3baa2499e201937 --project parquez
[mlrun] 2020-06-14 12:37:41,793 run executed, status=completed


In [308]:
# Run the data-ingestion function locally in Jupyter Notebook
#get_data_run = run_local(command= project.func('parquezrun'),workdir='./')

<a id="gs-step-create-n-run-ml-pipeline"></a>
## Create and Run a Fully Automated ML Pipeline

You're now ready to create a full ML pipeline.
This is done by using [Kubeflow Pipelines](https://www.kubeflow.org/docs/pipelines/overview/pipelines-overview/), which is integrated into the Iguazio Data Science Platform.
Kubeflow Pipelines is an open-source framework for building and deploying portable, scalable machine-learning workflows based on Docker containers.
MLRun leverages this framework to take your existing code and deploy it as steps in the pipeline.

In [309]:
%%writefile {path.join(project_path, 'workflow.py')}

from kfp import dsl
from mlrun import mount_v3io

funcs = {}

# Configure function resources and local settings
def init_functions(functions: dict, project=None, secrets=None):
    for f in functions.values():
        f.apply(mount_v3io())

    #functions['serving'].metadata.name = 'getting-started-serving'


# Create a Kubeflow Pipelines pipeline
@dsl.pipeline(
    name="parquez-pipeline",
    description="parquez description"
)
def kfpipeline():
    # Ingest the data set
    ingest = funcs['validate'].as_step(
        name="validate",
        params={'view_name':'view_name'
         ,'partition_by':'h'
         ,'partition_interval':'1h'
         ,'real_time_window':'1d'
         ,'historical_retention':'7d'
         ,'real_time_table_name':'faker'
         ,'config_path':'User/parquez/config/parquez.ini'}
        #handler='get_data',
        #inputs={'source_url': source_url},
        #params={'format': 'pq'},
        #outputs=[DATASET]
    )

    # Analyze the dataset
#     describe = funcs["describe"].as_step(
#         name="summary",
#         params={"label_column": LABELS},
#         inputs={"table": ingest.outputs[DATASET]})

#     # Train a model with hyperparemeters
#     train = funcs["train"].as_step(
#         name="train",
#         params={"sample": -1,
#                 "label_column": LABELS,
#                 "test_size": 0.10},
#         hyperparams={'model_pkg_class': MODELS},
#         selector='max.accuracy',
#         inputs={"dataset": ingest.outputs[DATASET]},
#         outputs=['model', 'test_set'])

#     # Test and visualize the model
#     test = funcs["test"].as_step(
#         name="test",
#         params={"label_column": LABELS},
#         inputs={"models_path": train.outputs['model'],
#                 "test_set": train.outputs['test_set']})

#     # Deploy the model as a serverless function
#     deploy = funcs["serving"].deploy_step(
#         models={f"{DATASET}_v1": train.outputs['model']})

#     # Test the new model server (via REST API calls)
#     tester = funcs["serving-tester"].as_step(
#         name='serving-tester',
#         params={'addr': deploy.outputs['endpoint'], 'model': f"{DATASET}_v1"},
#         inputs={'table': train.outputs['test_set']})

Overwriting /User/parquez/workflow.py


<a id="gs-register-workflow"></a>
#### Register the Workflow

Use the `set_workflow` MLRun project method to register your workflow with MLRun.
The following code sets the `name` parameter to the selected workflow name ("main") and the `code` parameter to the name of the workflow file that is found in your project directory (**workflow.py**).

In [310]:
# Register the workflow file as "main"
project.set_workflow('main', 'workflow.py')

In [311]:
project.save()

In [312]:
run_id = project.run(
    'main',
    arguments={}, 
    
    artifact_path=path.abspath(path.join('pipeline','{{workflow.uid}}'),
    
                              )
    ,dirty=True)

[mlrun] 2020-06-14 12:37:42,243 Pipeline run id=b3f46154-0cf8-4e26-a796-425d11d3a62d, check UI or DB for progress
